In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from os import listdir, getenv
import sqlalchemy as sa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Import the querying module
from flight_querying import query_flights
import pandas as pd

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [3]:
# Flight IDs to include
flight_ids = [4620, 4929, 4940, 5019, 5021, 5034]

# Fetch data for specified flight IDs
data_frames = [db_connect.connect_flight_for_ml_data_label(flight_id) for flight_id in flight_ids]

# Concatenate data frames and shuffle the data
all_data = pd.concat(data_frames, axis=0).sample(frac=1, random_state=42)

In [4]:

#multi-nomial encoding 
le = LabelEncoder()
all_data['y'] = le.fit_transform(all_data['exercise'])
print(all_data.head())

            time    id exercise  environment_temperature  dewpoint  humidity  \
4971   16.712741  5034       NA                     55.4      44.6     66.91   
57105  34.491229  4620  landing                     66.2      59.0     77.60   
25004   5.501430  4929       NA                     51.8      51.8    100.00   
33076   0.807816  5034       NA                     55.4      44.6     66.91   
48890  48.303963  5021       NA                     48.2      41.0     76.01   

       wind_speed  visibility  soc  cell_temperature  ...  motor_power  \
4971            9         9.0   96                17  ...            1   
57105           4         9.0   57                25  ...            0   
25004           2         5.0   99                15  ...            2   
33076           9         9.0  100                18  ...            0   
48890           3         9.0   35                12  ...            0   

       motor_temperature  indicated_air_speed  pressure_altitude  \
4971  

In [5]:
# Split the data into train and test sets
train_data, test_data = train_test_split(all_data, test_size=0.5, random_state=42)

# Set up train data
train_y = train_data["y"].to_numpy()
train_x = train_data.drop(columns=["exercise","id", "y"])

# Set up test data
test_y = test_data["y"].to_numpy()
test_x = test_data.drop(columns=["exercise", "id","y"])

In [6]:
print(f"Length train_x = {len(train_x)} \n Length train_y = {len(train_y)}")
print(f"Length test_x = {len(test_x)} \n Length test_y = {len(test_y)}")

Length train_x = 176069 
 Length train_y = 176069
Length test_x = 176069 
 Length test_y = 176069


In [7]:
train_x

,time,environment_temperature,dewpoint,humidity,wind_speed,visibility,soc,cell_temperature,motor_rpm,motor_power,motor_temperature,indicated_air_speed,pressure_altitude,ground_speed,outside_air_temperature,inverter_temperature,pitch,roll
672,2.346528,44.6,41.0,87.09,2,9.0,100,17,709,2,17.992535,0.000000,304.171631,0.0,11.0,22.463543,2.486865,-0.500044
27066,1.411336,75.2,57.2,53.55,3,9.0,100,21,0,0,25.748734,21.597849,309.338318,0.0,28.0,29.888712,2.836662,0.236644
21264,13.558906,53.6,41.0,62.23,7,9.0,85,19,2406,49,53.786613,81.975808,891.640381,89.3,9.0,36.152302,0.031875,-1.227109
7849,26.470125,48.2,41.0,76.01,3,9.0,65,15,1824,20,36.144962,69.054215,850.725159,57.1,8.0,27.939558,4.186932,18.071613
13681,46.235178,64.4,57.2,77.45,6,9.0,33,26,1134,0,51.103012,63.691637,452.486267,81.9,19.5,34.521172,-2.464786,3.403841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,5.252014,53.6,51.8,93.60,4,7.0,99,15,52,0,18.396080,0.000000,269.288208,0.0,12.5,23.230288,2.089734,-1.740982
41458,28.816188,55.4,44.6,66.91,9,9.0,70,17,2079,35,40.179089,65.504970,819.973083,63.5,9.5,31.172867,3.777274,1.198919
17140,8.499490,44.6,41.0,87.09,3,9.0,94,17,2229,44,41.405380,72.401552,504.710205,89.2,10.0,34.488026,9.151557,11.909757
34210,17.104419,44.6,41.0,87.09,2,9.0,74,16,1862,20,42.138077,74.121083,887.477722,62.8,7.0,30.351883,2.728413,-0.091229


In [8]:
train_y

array([0, 0, 0, ..., 1, 1, 3])

In [9]:
test_x

,time,environment_temperature,dewpoint,humidity,wind_speed,visibility,soc,cell_temperature,motor_rpm,motor_power,motor_temperature,indicated_air_speed,pressure_altitude,ground_speed,outside_air_temperature,inverter_temperature,pitch,roll
1249,4.298753,75.2,57.2,53.55,3,9.0,98,22,0,0,31.219154,24.340289,315.834106,0.0,27.0,33.765308,2.088632,0.022432
29238,8.668804,75.2,57.2,53.55,3,9.0,97,23,0,0,31.950922,22.442243,315.292511,0.0,26.5,35.332863,2.427966,-0.393822
60679,38.226257,57.2,44.6,62.69,10,9.0,52,16,2236,28,43.911201,92.886965,785.512329,93.1,10.5,31.511211,-1.359148,4.341249
59351,41.655112,66.2,59.0,77.60,4,9.0,44,26,2329,51,59.134201,71.703696,413.526031,80.9,20.0,40.505646,9.135372,0.779536
11643,19.536519,53.6,51.8,93.60,4,7.0,73,18,1975,21,44.334610,83.784373,769.117310,83.8,14.0,33.210754,-0.613944,-1.279893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49076,54.269230,55.4,44.6,66.91,9,9.0,35,17,599,1,31.371510,0.000000,307.058960,4.9,14.5,31.319542,3.403068,-0.432429
44225,32.570338,48.2,41.0,76.01,3,9.0,53,14,1693,20,39.639786,49.973051,1058.366577,38.2,6.5,29.932785,8.821197,-2.007098
14719,4.751320,75.2,57.2,53.55,4,9.0,98,22,0,0,31.390076,24.741226,315.930145,0.0,27.0,33.962784,2.260659,0.059773
29852,13.612635,51.8,51.8,100.00,2,5.0,82,18,1761,17,45.021774,69.156670,784.092773,70.4,13.5,33.148266,2.939765,-3.205949


In [10]:
test_y

array([0, 0, 2, ..., 0, 2, 2])

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [12]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(train_x, train_y)

/Users/nayeema/envs/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [13]:
# Make predictions on the test set
predictions = model.predict(test_x)

# Evaluate the model
accuracy = accuracy_score(test_y, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display additional classification metrics
print("Classification Report:")
print(classification_report(test_y, predictions))

/Users/nayeema/envs/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.75
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.85      0.89     63216
           1       0.78      0.79      0.79     17475
           2       0.64      0.90      0.75     42488
           3       0.47      0.26      0.34     13754
           4       0.71      0.72      0.71     18209
           5       0.58      0.85      0.69      1801
           6       0.00      0.00      0.00       641
           7       0.00      0.00      0.00      4579
           8       0.39      0.27      0.32      6421
           9       0.89      0.87      0.88      7485

    accuracy                           0.75    176069
   macro avg       0.54      0.55      0.54    176069
weighted avg       0.73      0.75      0.73    176069



/Users/nayeema/envs/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nayeema/envs/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Saving & Loading The Model

In [14]:
import joblib

# Assuming you've trained and named your Logistic Regression model 'logreg_model'

# Save the model to a file
model_filename = 'ML_model_outputs/multinomial_logreg_model.joblib'
joblib.dump(model, model_filename)

print(f"Model saved to {model_filename}")


Model saved to ML_model_outputs/multinomial_logreg_model.joblib


In [15]:
loaded_model = joblib.load(model_filename)